In [1]:
import numpy as np
from plyfile import PlyData, PlyElement
from tqdm import tqdm
from time import time
import cv2

In [2]:
plydata = PlyData.read('cloud_and_poses.ply')
plydata.elements[0].data

memmap([( 0.05679056, -6.08357455e-01, -0.05601962,   0, 255,   0),
        ( 0.05001685, -4.72463000e-01, -0.12602302,   0, 255,   0),
        ( 0.04550779, -3.20975554e-01, -0.18108519,   0, 255,   0),
        ( 0.04252427, -1.65877168e-01, -0.21379167,   0, 255,   0),
        ( 0.04409876, -6.71205668e-03, -0.22597731,   0, 255,   0),
        ( 0.2489538 , -4.04010331e-01,  2.05167315,   0, 255,   0),
        ( 0.24425407, -5.50674583e-01,  1.9865655 ,   0, 255,   0),
        ( 0.23509761, -6.86658978e-01,  1.90333359,   0, 255,   0),
        ( 0.22309037, -8.07659131e-01,  1.80425612,   0, 255,   0),
        ( 0.21116052, -9.14216827e-01,  1.68982638,   0, 255,   0),
        ( 0.19979522, -1.00634863e+00,  1.55918607,   0, 255,   0),
        ( 0.18856105, -1.07877136e+00,  1.41939239,   0, 255,   0),
        ( 0.18090507, -1.12772068e+00,  1.2814814 ,   0, 255,   0),
        ( 0.16493551, -1.16184230e+00,  1.13469792,   0, 255,   0),
        ( 0.15299045, -1.17605023e+00,  0.981314

In [3]:
camera_locations = plydata.elements[0].data[np.where(np.array(list(plydata.elements[0].data['red']))==0)[0]][["x","y","z"]]

In [4]:
camera_locations[0]

(0.05679056, -0.60835746, -0.05601962)

In [5]:
plydata

PlyData((PlyElement('vertex', (PlyProperty('x', 'double'), PlyProperty('y', 'double'), PlyProperty('z', 'double'), PlyProperty('red', 'uchar'), PlyProperty('green', 'uchar'), PlyProperty('blue', 'uchar')), count=801, comments=[]),), text=False, byte_order='<', comments=['generated by OpenMVG'], obj_info=[])

In [6]:
mesh = PlyData.read('./scene_dense_mesh_refine.ply')

In [7]:
mesh

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float')), count=3681, comments=[]), PlyElement('face', (PlyListProperty('vertex_indices', 'uchar', 'uint'),), count=7314, comments=[])), text=False, byte_order='<', comments=[], obj_info=[])

In [8]:
def fetch_mesh_coordinates(mesh_coords):
    p1 = np.array(list(mesh.elements[0].data[mesh_coords[0][0]]))
    p2 = np.array(list(mesh.elements[0].data[mesh_coords[0][1]]))
    p3 = np.array(list(mesh.elements[0].data[mesh_coords[0][2]]))
    a1 = p2-p1
    a2 = p3-p1
    cross_product = np.cross(a1,a2)
    norm_cross_product = np.linalg.norm(cross_product)
    #Normal vector
    n =  cross_product/norm_cross_product
    #Area
    area = (1/2)*norm_cross_product
    return ((p1,p2,p3),n,area)

def SignedVolume(a,b,c,d):
    return (1.0/6.0)*np.dot(np.cross(b-a,c-a),d-a)

def intersecting_mesh(p1,p2,p3,q1,q2):
    """Finds if vector formed by q1,q2 intersects the triangle made by p1,p2,p3. Returns 1 if there is intersection else 0"""
    A1 = np.sign(SignedVolume(q1,p1,p2,p3))
    A2 = np.sign(SignedVolume(q2,p1,p2,p3))
    B1 = np.sign(SignedVolume(q1,q2,p1,p2))
    B2 = np.sign(SignedVolume(q1,q2,p2,p3))
    B3 = np.sign(SignedVolume(q1,q2,p3,p1))
    if (A1!=A2) and (B1==B2==B3):
        return 1
    else:
        return 0

In [9]:
Mesh_info = []
for i in tqdm(range(len(mesh.elements[1].data))):
    Mesh_info.append(fetch_mesh_coordinates(mesh.elements[1].data[i]))

100%|██████████| 7314/7314 [00:01<00:00, 4618.40it/s]


In [10]:
Mesh_vertices = np.array([[i[0][0],i[0][1],i[0][2]]for i in Mesh_info])
mesh_centroid = np.mean(Mesh_vertices,axis=1)

In [11]:
def faster_visibility(mesh_index,camera_loc):
    (p1,p2,p3),n,area=Mesh_info[mesh_index]
    q1 = camera_loc
    q2 = (p1+p2+p3)/3
    vector = vector = (q2-q1)/np.linalg.norm(q2-q1)
    #Calculates points on the vector joining
    Mesh_vector = q1+ np.array([[list(np.linalg.norm((mesh_centroid-q1),axis=1)),]*3]).T[:,:,0]*vector
    dist = np.linalg.norm(np.mean(Mesh_vertices[mesh_index,:],axis=0)-Mesh_vertices[mesh_index,0])    
    Search_mesh = np.where((np.abs(Mesh_vector[:,0]-mesh_centroid[:,0])<=dist) & (np.abs(Mesh_vector[:,1]-mesh_centroid[:,1])<=dist) & (np.abs(Mesh_vector[:,2]-mesh_centroid[:,2])<=dist))[0] 
    #Modifying q2 so that it is just outside the triangle
    q2 = q2 + 0.00001*(q2-q1)/np.linalg.norm(q2-q1)
    #Visibility function
    visibility = []
    main_mesh = mesh_index
    for mesh_index in Search_mesh:
        (p1,p2,p3),n,area = Mesh_info[mesh_index]
        visibility.append(intersecting_mesh(p1,p2,p3,q1,q2))
    a = Search_mesh[np.where(np.array(visibility)==1)[0]]
    if len(a)>=2:
        return 0
    else: 
        return 1

In [12]:
camera_loc = np.array([0.04409876, -6.71205668e-03, -0.22597731])
mesh_index = 5168
faster_visibility(mesh_index,camera_loc)

0

In [13]:
start = time()
Mesh_visibility = np.zeros(len(mesh_centroid))
Mesh_list = np.arange(len(mesh_centroid))
camera_loc = np.array(list(camera_locations[0]))
Mesh_vertices_calc = Mesh_vertices.copy()
while len(Mesh_list)>=1:
    print("{} meshes left....".format(len(Mesh_list)))
    mesh_index = Mesh_list[0]
    a = faster_visibility(mesh_index,camera_loc)
    #Find neighbouring meshes
    indices = np.unique(np.where((Mesh_vertices_calc==Mesh_vertices[mesh_index][0]) | (Mesh_vertices_calc==Mesh_vertices[mesh_index][1]) | (Mesh_vertices_calc==Mesh_vertices[mesh_index][2]))[0])
    #Update the visibility as the one of the selected mesh   
    Mesh_visibility[Mesh_list[indices]] = a
    #Update the mesh list
    Mesh_list = np.delete(Mesh_list, indices)
    #Update the vertices list
    Mesh_vertices_calc = np.delete(Mesh_vertices_calc, indices,axis=0)
end = time()
print(end-start)

7314 meshes left....
7300 meshes left....
7286 meshes left....
7272 meshes left....
7258 meshes left....
7250 meshes left....
7239 meshes left....
7229 meshes left....
7214 meshes left....
7199 meshes left....
7188 meshes left....
7174 meshes left....
7160 meshes left....
7146 meshes left....
7132 meshes left....
7119 meshes left....
7106 meshes left....
7095 meshes left....
7080 meshes left....
7067 meshes left....
7053 meshes left....
7043 meshes left....
7028 meshes left....
7014 meshes left....
7000 meshes left....
6987 meshes left....
6973 meshes left....
6962 meshes left....
6954 meshes left....
6940 meshes left....
6924 meshes left....
6910 meshes left....
6898 meshes left....
6883 meshes left....
6869 meshes left....
6856 meshes left....
6838 meshes left....
6826 meshes left....
6813 meshes left....
6802 meshes left....
6788 meshes left....
6776 meshes left....
6764 meshes left....
6750 meshes left....
6741 meshes left....
6727 meshes left....
6713 meshes left....
6698 meshes l

3195 meshes left....
3180 meshes left....
3166 meshes left....
3160 meshes left....
3152 meshes left....
3145 meshes left....
3136 meshes left....
3126 meshes left....
3113 meshes left....
3101 meshes left....
3092 meshes left....
3082 meshes left....
3073 meshes left....
3067 meshes left....
3065 meshes left....
3054 meshes left....
3052 meshes left....
3039 meshes left....
3032 meshes left....
3026 meshes left....
3013 meshes left....
3002 meshes left....
2995 meshes left....
2989 meshes left....
2985 meshes left....
2975 meshes left....
2962 meshes left....
2949 meshes left....
2943 meshes left....
2932 meshes left....
2917 meshes left....
2906 meshes left....
2897 meshes left....
2887 meshes left....
2877 meshes left....
2874 meshes left....
2864 meshes left....
2855 meshes left....
2845 meshes left....
2838 meshes left....
2834 meshes left....
2822 meshes left....
2816 meshes left....
2805 meshes left....
2796 meshes left....
2782 meshes left....
2773 meshes left....
2762 meshes l

642 meshes left....
641 meshes left....
640 meshes left....
636 meshes left....
629 meshes left....
628 meshes left....
624 meshes left....
622 meshes left....
618 meshes left....
613 meshes left....
612 meshes left....
606 meshes left....
603 meshes left....
601 meshes left....
598 meshes left....
592 meshes left....
590 meshes left....
587 meshes left....
585 meshes left....
578 meshes left....
577 meshes left....
571 meshes left....
570 meshes left....
564 meshes left....
561 meshes left....
559 meshes left....
558 meshes left....
557 meshes left....
555 meshes left....
551 meshes left....
547 meshes left....
540 meshes left....
536 meshes left....
528 meshes left....
527 meshes left....
525 meshes left....
519 meshes left....
514 meshes left....
510 meshes left....
508 meshes left....
506 meshes left....
503 meshes left....
496 meshes left....
488 meshes left....
483 meshes left....
482 meshes left....
481 meshes left....
478 meshes left....
476 meshes left....
475 meshes left....


In [13]:
start = time()
Camera_visibility = []
for camera in tqdm(range(len(camera_locations))):
    Mesh_visibility = np.zeros(len(mesh_centroid))
    Mesh_list = np.arange(len(mesh_centroid))
    camera_loc = np.array(list(camera_locations[camera]))
    Mesh_vertices_calc = Mesh_vertices.copy()
    while len(Mesh_list)>=1:
#         print("{} meshes left....".format(len(Mesh_list)))
        mesh_index = Mesh_list[0]
        a = faster_visibility(mesh_index,camera_loc)
        #Find neighbouring meshes
        indices = np.unique(np.where((Mesh_vertices_calc==Mesh_vertices[mesh_index][0]) | (Mesh_vertices_calc==Mesh_vertices[mesh_index][1]) | (Mesh_vertices_calc==Mesh_vertices[mesh_index][2]))[0])
        #Update the visibility as the one of the selected mesh   
        Mesh_visibility[Mesh_list[indices]] = a
        #Update the mesh list
        Mesh_list = np.delete(Mesh_list, indices)
        #Update the vertices list
        Mesh_vertices_calc = np.delete(Mesh_vertices_calc, indices,axis=0)
    Camera_visibility.append(Mesh_visibility)
end = time()
print(end-start)

100%|██████████| 24/24 [09:52<00:00, 24.69s/it]

592.561007976532


In [16]:
Mesh_visibility[6946]

1.0

# Energy Function

In [17]:
import json

In [19]:
f1 = open("camera_params.test", "r")
data_RC = json.load(f1)['params']


f2 = open("images.test", "r")
data_P = json.load(f2)['data']

def valid(l):
    for x in l:
        if x != 0:
            return True
    return False

data_P = [x for x in data_P if x['height'] != 0 and valid(x['P']) ]
files = [x['name'] for x in data_P]

# Camera details
P = np.array([np.array(x['P']).reshape([3, 4]) for x in data_P])
R = np.array([np.array(x['R']).reshape([3, 3]) for x in data_RC])
C = np.array([np.array(x['C']) for x in data_RC])
K = np.matmul(P[:, :, :3], np.linalg.inv(R))


In [25]:
import time
points = []
sample_size = 14
for u in np.arange(0,1,1/sample_size):
    for v in np.arange(0,1-u,1/sample_size):
        w = 1-u-v
        points.append([u,v,w])
points =  np.array(points)

In [26]:
camera_index = 2
f_x = K[camera_index][0,0]
f_y = K[camera_index][1,1]
visibility = Camera_visibility[camera_index]
P_camera = P[camera_index]
C = np.array(list(camera_locations[camera_index]))
#Load all images for each camera
Images = [cv2.imread("./datasets/templeRing/"+files[i].split("/")[-1]) for i in range(len(camera_locations))]
start = time.time()
#For a single camera
integration = 0
# mesh_id = 6946
for mesh_id in tqdm(range(len(Mesh_info))):
    X_j = Mesh_vertices[mesh_id] 
    _,n_j,A_j = Mesh_info[mesh_id]
    #P=uA+vB+wC
    X_u = np.dot(points,X_j)
    se = np.dot(P_camera,np.hstack((X_u,np.ones((len(X_u),1)))).T).T
    se = np.divide(se,se[:,-1][:,np.newaxis])
    #Texture term shd be added
    image_term = np.linalg.norm(Images[camera_index][se[:,1].astype(int),se[:,0].astype(int)],axis=1)
    d = X_u - C
    d_z = np.linalg.norm(d,axis=1)
    alpha = (f_x*f_y)*np.divide(d,d_z[:,np.newaxis]**3)
    integration += A_j*np.dot(image_term,np.dot(alpha,n_j))*visibility[mesh_id]
print(integration)
end = time.time()
print(end-start)

100%|██████████| 7314/7314 [00:01<00:00, 5010.59it/s]

-1834416166.5392764
1.4645097255706787


In [ ]:
#Load all images for each camera
Images = [cv2.imread("./datasets/templeRing/"+files[i].split("/")[-1]) for i in range(len(camera_locations))]

In [36]:
def Energy_function_calc(sample_size=14):
    #Calculates barycentric coordinates
    points = []
    for u in np.arange(0,1,1/sample_size):
        for v in np.arange(0,1-u,1/sample_size):
            w = 1-u-v
            points.append([u,v,w])
    points = np.array(points)
    h,w,_ = np.shape(Images[0])
    integration = 0
    
    for camera_index in tqdm(range(len(camera_locations))):
        f_x = K[camera_index][0,0]
        f_y = K[camera_index][1,1]
        visibility = Camera_visibility[camera_index]
        P_camera = P[camera_index]
        C = np.array(list(camera_locations[camera_index]))
        for mesh_id in range(len(Mesh_info)):
            X_j = Mesh_vertices[mesh_id] 
            _,n_j,A_j = Mesh_info[mesh_id]
            #P=uA+vB+wC
            X_u = np.dot(points,X_j)
            se = np.dot(P_camera,np.hstack((X_u,np.ones((len(X_u),1)))).T).T
            se = np.divide(se,se[:,-1][:,np.newaxis])
            #Texture term shd be added
            image_term = np.linalg.norm(Images[camera_index][np.clip(se[:,1].astype(int),0,h-1),np.clip(se[:,0].astype(int),0,w-1)],axis=1)
            d = X_u - C
            d_z = np.linalg.norm(d,axis=1)
            alpha = (f_x*f_y)*np.divide(d,d_z[:,np.newaxis]**3)
            integration += (10**-6)*A_j*np.dot(image_term,np.dot(alpha,n_j))*visibility[mesh_id]
    return integration

In [37]:
integeration = Energy_function_calc()
integeration

100%|██████████| 24/24 [01:05<00:00,  2.72s/it]


-46858.880873645605

In [46]:
points = []
for u in np.arange(0,1,1/sample_size):
    for v in np.arange(0,1-u,1/sample_size):
        w = 1-u-v
        points.append([u,v,w])
points = np.array(points)

In [138]:

mesh_id = 6946
X_j = Mesh_vertices[mesh_id] 
_,n_j,A_j = Mesh_info[mesh_id]
#P=uA+vB+wC
X_u = np.dot(points,X_j)
se = np.dot(P_camera,np.hstack((X_u,np.ones((len(X_u),1)))).T).T
se = np.divide(se,se[:,-1][:,np.newaxis])
#Texture term shd be added
image_term = np.linalg.norm(Images[camera_index][se[:,1].astype(int),se[:,0].astype(int)],axis=1)
d = X_u - C
d_z = np.linalg.norm(d,axis=1)
alpha = (f_x*f_y)*np.divide(d,d_z[:,np.newaxis]**3)
integration += A_j*np.dot(image_term,np.dot(alpha,n_j))*visibility[mesh_id]
print(integration)



-816402.8188562718
0.003964662551879883


In [142]:
camera_index = 2
f_x = K[camera_index][0,0]
f_y = K[camera_index][1,1]
visibility = Camera_visibility[camera_index]
P_camera = P[camera_index]
C = np.array(list(camera_locations[camera_index]))
#Load all images for each camera
Images = [cv2.imread("./datasets/templeRing/"+files[i].split("/")[-1]) for i in range(len(camera_locations))]
start = time.time()
#For a single camera
integration = 0
# mesh_id = 6946
for mesh_id in tqdm(range(len(Mesh_info))):
    sample_size = 14 #Generates N*(N+1)/2 triplets
    X_j = Mesh_vertices[mesh_id] 
    _,n_j,A_j = Mesh_info[mesh_id]
    for u in np.arange(0,1,1/sample_size):
        for v in np.arange(0,1-u,1/sample_size):
            w = 1-u-v
            #P=uA+vB+wC
            x_u = np.dot(np.array([u,v,w]),X_j)
            se = np.dot(P_camera,np.hstack((x_u,1)).T)
            se = se/se[-1]
            #Texture term shd be added
            image_term = np.linalg.norm(Images[camera_index][int(se[1]),int(se[0])])
            #Alpha term
            d = x_u - C
            d_z = np.linalg.norm(x_u-C)
            alpha = (f_x*f_y/(d_z**3))*d
            integration+=image_term*np.dot(alpha,n_j)*visibility[mesh_id]
    integeration += A_j*integration
print(integeration)
end = time.time()
print(end-start)

100%|██████████| 7314/7314 [01:57<00:00, 62.39it/s]

-6902759485933.656
117.22951340675354


# Rough

In [ ]:
Mesh_visibility[1505]
camera_loc = np.array([0.04409876, -6.71205668e-03, -0.22597731])
mesh_index = 5230
centroid = np.mean(Mesh_vertices[5230,:],axis=0)
dist = np.linalg.norm(np.mean(Mesh_vertices[5230,:],axis=0)-Mesh_vertices[5230,0])

In [ ]:
def visibility_function(mesh_index,camera_loc):
    (p1,p2,p3),n,area=Mesh_info[mesh_index]
    q1 = camera_loc
    q2 = (p1+p2+p3)/3
    #Modifying q2 so that it is just outside the triangle
    q2 = q2 + 0.00001*(q2-q1)/np.linalg.norm(q2-q1)
    #Visibility function
    visibility = []
    main_mesh = mesh_index
    for mesh_index in tqdm(range(len(mesh.elements[1].data))):
        (p1,p2,p3),n,area = Mesh_info[mesh_index]
        visibility.append(intersecting_mesh(p1,p2,p3,q1,q2))
    a = np.where(np.array(visibility)==1)[0]
    intermediate_meshes = a[np.where(a!=main_mesh)]
    return intermediate_meshes

In [78]:
from __future__ import print_function
from CGAL.CGAL_Kernel import Point_3
from CGAL.CGAL_Kernel import Vector_3
from CGAL.CGAL_Kernel import Plane_3
from CGAL.CGAL_Kernel import Segment_3
from CGAL.CGAL_Polyhedron_3 import Polyhedron_3
from CGAL.CGAL_AABB_tree import AABB_tree_Polyhedron_3_Facet_handle


p = Point_3(1.0, 0.0, 0.0)
q = Point_3(0.0, 1.0, 0.0)
r = Point_3(0.0, 0.0, 1.0)
s = Point_3(0.0, 0.0, 0.0)
polyhedron = Polyhedron_3()
polyhedron.make_tetrahedron(p, q, r, s)

# constructs AABB tree
tree = AABB_tree_Polyhedron_3_Facet_handle(polyhedron.facets())

# constructs segment query
a = Point_3(-0.2, 0.2, -0.2)
b = Point_3(1.3, 0.2, 1.3)
segment_query = Segment_3(a, b)

# tests intersections with segment query
if tree.do_intersect(segment_query):
    print("intersection(s)")
else:
    print("no intersection")

# computes #intersections with segment query
print(tree.number_of_intersected_primitives(segment_query), " intersection(s)")

# computes first encountered intersection with segment query
# (generally a point)
intersection = tree.any_intersection(segment_query)
if not intersection.empty():
    # gets intersection object
    op = intersection.value()
    object = op[0]
    if object.is_Point_3():
        print("intersection object is a point")


# computes all intersections with segment query (as pairs object - primitive_id)
intersections = []
tree.all_intersections(segment_query, intersections)

# computes all intersected primitives with segment query as primitive ids
primitives = []
tree.all_intersected_primitives(segment_query, primitives)

# constructs plane query
vec = Vector_3(0.0, 0.0, 1.0)
plane_query = Plane_3(a, vec)

# computes first encountered intersection with plane query
# (generally a segment)
intersection = tree.any_intersection(plane_query)
if not intersection.empty():
    # gets intersection object
    op = intersection.value()
    object = op[0]
    if object.is_Segment_3():
        print("intersection object is a segment")

AttributeError: module 'CGAL._CGAL_Kernel' has no attribute 'ON_NEGATIVE_SIDE'

In [171]:
mesh_index = 5230

In [165]:
#Mesh visibility
camera_loc = np.array([0.04409876, -6.71205668e-03, -0.22597731])
mesh_index = 5230
(p1,p2,p3),n,area=Mesh_info[mesh_index]
q1 = camera_loc
q2 = (p1+p2+p3)/3
#Modifying q2 so that it is just outside the triangle
q2 = q2 + 0.00001*(q2-q1)/np.linalg.norm(q2-q1)
intersecting_mesh(p1,p2,p3,q1,q2)

1

In [166]:
#Visibility function
visibility = []
main_mesh = 5230
for mesh_index in tqdm(range(len(mesh.elements[1].data))):
    (p1,p2,p3),n,area = Mesh_info[mesh_index]
    visibility.append(intersecting_mesh(p1,p2,p3,q1,q2))

100%|██████████| 7314/7314 [00:04<00:00, 1759.85it/s]


In [167]:
a = np.where(np.array(visibility)==1)[0]
a[np.where(a!=main_mesh)]

array([6567])

In [137]:
Mesh_info[0]

((array([-0.06684893, -0.04645758,  0.9324059 ], dtype=float32),
  array([-0.0770964 , -0.04810393,  0.9338913 ], dtype=float32),
  array([-0.07262716, -0.0410408 ,  0.93799603], dtype=float32)),
 array([-0.20770185,  0.5864206 , -0.78292453], dtype=float32),
 4.152459223405458e-05)

In [139]:
p1,p2,p3,n,area = Mesh_info[0]

ValueError: not enough values to unpack (expected 5, got 3)

In [50]:
import numpy as np
from plyfile import PlyData, PlyElement
from tqdm import tqdm
from time import time
import json
import cv2

######################################### FUNCTIONS ###########################################################
def valid(l):
    for x in l:
        if x != 0:
            return True
    return False

def return_camera_info(file1,file2):
    '''
    Reads json file and returns camera parameters and locations
    '''
    f1 = open(file1, "r")
    data_RC = json.load(f1)['params']

    f2 = open(file2, "r")
    data_P = json.load(f2)['data']

    data_P = [x for x in data_P if x['height'] != 0 and valid(x['P']) ]
    files = [x['name'] for x in data_P]

    # Camera details
    P = np.array([np.array(x['P']).reshape([3, 4]) for x in data_P])
    R = np.array([np.array(x['R']).reshape([3, 3]) for x in data_RC])
    C = np.array([np.array(x['C']) for x in data_RC])
    K = np.matmul(P[:, :, :3], np.linalg.inv(R))

    return (P,R,C,K,files)

def fetch_mesh_coordinates(mesh_coords):
    '''
    Fetches coordinates,normal,area of the meshes
    '''
    p1 = np.array(list(mesh.elements[0].data[mesh_coords[0][0]]))
    p2 = np.array(list(mesh.elements[0].data[mesh_coords[0][1]]))
    p3 = np.array(list(mesh.elements[0].data[mesh_coords[0][2]]))
    a1 = p2-p1
    a2 = p3-p1
    cross_product = np.cross(a1,a2)
    norm_cross_product = np.linalg.norm(cross_product)
    #Normal vector
    n =  cross_product/norm_cross_product
    #Area
    area = (1/2)*norm_cross_product
    return ((p1,p2,p3),n,area)

def fetch_area_normal(pts):
    '''
    Fetches area and normal of the triangle formed by the points
    '''
    p1 = pts[0]
    p2 = pts[1]
    p3 = pts[2]
    a1 = p2-p1
    a2 = p3-p1
    cross_product = np.cross(a1,a2)
    norm_cross_product = np.linalg.norm(cross_product)
    #Normal vector
    n =  cross_product/norm_cross_product
    #Area
    area = (1/2)*norm_cross_product
    return (n,area)

def SignedVolume(a,b,c,d):
    return (1.0/6.0)*np.dot(np.cross(b-a,c-a),d-a)

def intersecting_mesh(p1,p2,p3,q1,q2):
    """Finds if vector formed by q1,q2 intersects the triangle made by p1,p2,p3. Returns 1 if there is intersection else 0"""
    A1 = np.sign(SignedVolume(q1,p1,p2,p3))
    A2 = np.sign(SignedVolume(q2,p1,p2,p3))
    B1 = np.sign(SignedVolume(q1,q2,p1,p2))
    B2 = np.sign(SignedVolume(q1,q2,p2,p3))
    B3 = np.sign(SignedVolume(q1,q2,p3,p1))
    if (A1!=A2) and (B1==B2==B3):
        return 1
    else:
        return 0
    
def visibility(mesh_index,camera_loc):
    (p1,p2,p3),n,area=Mesh_info[mesh_index]
    q1 = camera_loc
    q2 = (p1+p2+p3)/3
    vector = vector = (q2-q1)/np.linalg.norm(q2-q1)
    #Calculates points on the vector joining
    Mesh_vector = q1+ np.array([[list(np.linalg.norm((mesh_centroid-q1),axis=1)),]*3]).T[:,:,0]*vector
    dist = np.linalg.norm(np.mean(Mesh_vertices[mesh_index,:],axis=0)-Mesh_vertices[mesh_index,0])    
    Search_mesh = np.where((np.abs(Mesh_vector[:,0]-mesh_centroid[:,0])<=dist) & (np.abs(Mesh_vector[:,1]-mesh_centroid[:,1])<=dist) & (np.abs(Mesh_vector[:,2]-mesh_centroid[:,2])<=dist))[0] 
    #Modifying q2 so that it is just outside the triangle
    q2 = q2 + 0.00001*(q2-q1)/np.linalg.norm(q2-q1)
    #Visibility function
    visibility = []
    main_mesh = mesh_index
    for mesh_index in Search_mesh:
        (p1,p2,p3),n,area = Mesh_info[mesh_index]
        visibility.append(intersecting_mesh(p1,p2,p3,q1,q2))
    a = Search_mesh[np.where(np.array(visibility)==1)[0]]
    if len(a)>=2:
        return 0
    else: 
        return 1

def calculate_visibility_mesh(camera_locations):
    start = time()
    Camera_visibility = []
    for camera in tqdm(range(len(camera_locations)),desc="Calculating visibility for given list of {} cameras".format(len(camera_locations))):
        Mesh_visibility = np.zeros(len(mesh_centroid))
        Mesh_list = np.arange(len(mesh_centroid))
        camera_loc = np.array(list(camera_locations[camera]))
        Mesh_vertices_calc = Mesh_vertices.copy()
        while len(Mesh_list)>=1:
    #         print("{} meshes left....".format(len(Mesh_list)))
            mesh_index = Mesh_list[0]
            a = visibility(mesh_index,camera_loc)
            #Find neighbouring meshes
            indices = np.unique(np.where((Mesh_vertices_calc==Mesh_vertices[mesh_index][0]) | (Mesh_vertices_calc==Mesh_vertices[mesh_index][1]) | (Mesh_vertices_calc==Mesh_vertices[mesh_index][2]))[0])
            #Update the visibility as the one of the selected mesh   
            Mesh_visibility[Mesh_list[indices]] = a
            #Update the mesh list
            Mesh_list = np.delete(Mesh_list, indices)
            #Update the vertices list
            Mesh_vertices_calc = np.delete(Mesh_vertices_calc, indices,axis=0)
        Camera_visibility.append(Mesh_visibility)
    end = time()
    print("Done in {}sec".format(end-start))
    return Camera_visibility

def Energy_function_calc(sample_size=14):
    '''Finds Energy value for the whole mesh and camera'''
    #Calculates barycentric coordinates
    points = []
    for u in np.arange(0,1,1/sample_size):
        for v in np.arange(0,1-u,1/sample_size):
            w = 1-u-v
            points.append([u,v,w])
    points = np.array(points)
    h,w,_ = np.shape(Images[0])
    integration = 0
    Energy_over_mesh = np.zeros(len(Mesh_info))
    
    for camera_index in tqdm(range(len(camera_locations)),desc="Calculating energy function..."):
        f_x = K[camera_index][0,0]
        f_y = K[camera_index][1,1]
        visibility = Camera_visibility[camera_index]
        P_camera = P[camera_index]
        C = np.array(list(camera_locations[camera_index]))
        for mesh_id in range(len(Mesh_info)):
            if visibility[mesh_id]!=0:    
                X_j = Mesh_vertices[mesh_id] 
                _,n_j,A_j = Mesh_info[mesh_id]
                #P=uA+vB+wC
                X_u = np.dot(points,X_j)
                #Finds x = PX
                se = np.dot(P_camera,np.hstack((X_u,np.ones((len(X_u),1)))).T).T
                se = np.divide(se,se[:,-1][:,np.newaxis])
                #Texture term shd be added, term from image fetched
                image_term = np.linalg.norm(Images[camera_index][np.clip(se[:,1].astype(int),0,h-1),np.clip(se[:,0].astype(int),0,w-1)],axis=1)
                #For alpha term
                d = X_u - C
                d_z = np.linalg.norm(d,axis=1)
                alpha = (10**-6)*(f_x*f_y)*np.divide(d,d_z[:,np.newaxis]**3)
                #Final integeration over a single meshe
                temp_val = A_j*np.dot(image_term,np.dot(alpha,n_j))*visibility[mesh_id]
                Energy_over_mesh[mesh_id] = Energy_over_mesh[mesh_id] + temp_val
                integration += temp_val
    return integration,Energy_over_mesh
#########################################################################################################################


In [49]:
#Fetch initial scene data
P,R,camera_locations,K,files = return_camera_info("camera_params.test","images.test")
mesh = PlyData.read('./scene_dense_mesh_refine.ply')
#Load all images for each camera
operating_dir = "./datasets/templeRing/"
Images = [cv2.imread(operating_dir+files[i].split("/")[-1]) for i in range(len(camera_locations))]

#Extract mesh related information
Mesh_info = []
for i in tqdm(range(len(mesh.elements[1].data)),desc="Fetching mesh related information..."):
    Mesh_info.append(fetch_mesh_coordinates(mesh.elements[1].data[i]))
#Coordinates of vertex each mesh
Mesh_vertices = np.array([[i[0][0],i[0][1],i[0][2]]for i in Mesh_info])
#Centroid of mesh
mesh_centroid = np.mean(Mesh_vertices,axis=1)

#Visibility table
Camera_visibility = calculate_visibility_mesh(camera_locations)

integeration,Energy_over_mesh = Energy_function_calc()
print(integeration)

Fetching mesh related information...: 100%|██████████| 7314/7314 [00:01<00:00, 4302.43it/s]
Calculating visibility for given list of 24 cameras: 100%|██████████| 24/24 [10:35<00:00, 26.49s/it]
Calculating energy function...:   0%|          | 0/24 [00:00<?, ?it/s]

Done in 635.7971134185791sec


NameError: name 'temp' is not defined

In [51]:
integeration,Energy_over_mesh = Energy_function_calc()

Calculating energy function...: 100%|██████████| 24/24 [00:32<00:00,  1.36s/it]


In [4]:
Vertex = np.array([np.array(list(mesh.elements[0].data[i])) for i in range(len(mesh.elements[0].data))])

In [5]:
#Numerical gradient calculation
def Numerical_gradient_mesh(vertex_id,diff=0.0000001,sample_size=14):
    '''Calculates numerical gradient'''
    #Calculates barycentric coordinates
    points = []
    for u in np.arange(0,1,1/sample_size):
        for v in np.arange(0,1-u,1/sample_size):
            w = 1-u-v
            points.append([u,v,w])
    points = np.array(points)
    h,w,_ = np.shape(Images[0])
    affected_meshes = np.unique(np.where((Mesh_vertices==Vertex[vertex_id]))[0])
    gradient = np.array([0.0,0.0,0.0]) 
    for axis in range(3):
        Vertex_gradient = Vertex.copy()
        integration = 0
        for camera_index in range(len(camera_locations)):
            f_x = K[camera_index][0,0]
            f_y = K[camera_index][1,1]
            visibility = Camera_visibility[camera_index]
            P_camera = P[camera_index]
            C = np.array(list(camera_locations[camera_index]))
            for mesh_id in affected_meshes:
                #Positive perturbation f(x+diff)
                if visibility[mesh_id]!=0:    
                    Vertex_gradient[vertex_id][axis] = Vertex[vertex_id][axis]+diff
                    X_j = Vertex_gradient[mesh.elements[1].data[mesh_id][0]] 
                    n_j,A_j = fetch_area_normal(X_j)
                    #P=uA+vB+wC
                    X_u = np.dot(points,X_j)
                    #Finds x = PX
                    se = np.dot(P_camera,np.hstack((X_u,np.ones((len(X_u),1)))).T).T
                    se = np.divide(se,se[:,-1][:,np.newaxis])
                    #Texture term shd be added, term from image fetched
                    image_term = np.linalg.norm(Images[camera_index][np.clip(se[:,1].astype(int),0,h-1),np.clip(se[:,0].astype(int),0,w-1)],axis=1)
                    #For alpha term
                    d = X_u - C
                    d_z = np.linalg.norm(d,axis=1)
                    alpha = (10**-7)*(f_x*f_y)*np.divide(d,d_z[:,np.newaxis]**3)
                    #Final integeration over a single meshe
                    integration_pos = A_j*np.dot(image_term,np.dot(alpha,n_j))*visibility[mesh_id]

                    #Negative perturbation f(x-diff)
                    Vertex_gradient[vertex_id][axis] = Vertex[vertex_id][axis]-diff
                    X_j = Vertex_gradient[mesh.elements[1].data[mesh_id][0]] 
                    n_j,A_j = fetch_area_normal(X_j)
                    #P=uA+vB+wC
                    X_u = np.dot(points,X_j)
                    #Finds x = PX
                    se = np.dot(P_camera,np.hstack((X_u,np.ones((len(X_u),1)))).T).T
                    se = np.divide(se,se[:,-1][:,np.newaxis])
                    #Texture term shd be added, term from image fetched
                    image_term = np.linalg.norm(Images[camera_index][np.clip(se[:,1].astype(int),0,h-1),np.clip(se[:,0].astype(int),0,w-1)],axis=1)
                    #For alpha term
                    d = X_u - C
                    d_z = np.linalg.norm(d,axis=1)
                    alpha = (10**-7)*(f_x*f_y)*np.divide(d,d_z[:,np.newaxis]**3)
                    #Final integeration over a single meshe
                    integration_neg = A_j*np.dot(image_term,np.dot(alpha,n_j))*visibility[mesh_id]

                    #Final addition 
                    integration += (integration_pos-integration_neg)
        gradient[axis] = integration/(2*diff)
    return gradient

In [6]:
Numerical_gradient_mesh(698)

array([-197.41772445,  -40.26300969,  -96.80567865])

In [51]:
Vertex[20]

array([-0.1188213 , -0.05593536,  0.9343591 ], dtype=float32)

In [64]:
np.where((Mesh_vertices==Vertex[23]))

(array([ 236,  236,  236, 1961, 1961, 1961, 1963, 1963, 1963, 1972, 1972,
        1972, 1978, 1978, 1978, 1979, 1979, 1979]),
 array([2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2]),
 array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]))

In [56]:
Mesh_vertices[6164]

array([[-0.1188213 , -0.05593536,  0.9343591 ],
       [-0.12459865, -0.05923643,  0.9377508 ],
       [-0.12685685, -0.05451554,  0.9348972 ]], dtype=float32)

In [57]:
Vertex[mesh.elements[1].data[6164][0]] 

array([[-0.1188213 , -0.05593536,  0.9343591 ],
       [-0.12459865, -0.05923643,  0.9377508 ],
       [-0.12685685, -0.05451554,  0.9348972 ]], dtype=float32)

In [13]:
plydata = PlyData.read('cloud_and_poses.ply')
plydata.elements[0].data

memmap([( 0.05679056, -6.08357455e-01, -0.05601962,   0, 255,   0),
        ( 0.05001685, -4.72463000e-01, -0.12602302,   0, 255,   0),
        ( 0.04550779, -3.20975554e-01, -0.18108519,   0, 255,   0),
        ( 0.04252427, -1.65877168e-01, -0.21379167,   0, 255,   0),
        ( 0.04409876, -6.71205668e-03, -0.22597731,   0, 255,   0),
        ( 0.2489538 , -4.04010331e-01,  2.05167315,   0, 255,   0),
        ( 0.24425407, -5.50674583e-01,  1.9865655 ,   0, 255,   0),
        ( 0.23509761, -6.86658978e-01,  1.90333359,   0, 255,   0),
        ( 0.22309037, -8.07659131e-01,  1.80425612,   0, 255,   0),
        ( 0.21116052, -9.14216827e-01,  1.68982638,   0, 255,   0),
        ( 0.19979522, -1.00634863e+00,  1.55918607,   0, 255,   0),
        ( 0.18856105, -1.07877136e+00,  1.41939239,   0, 255,   0),
        ( 0.18090507, -1.12772068e+00,  1.2814814 ,   0, 255,   0),
        ( 0.16493551, -1.16184230e+00,  1.13469792,   0, 255,   0),
        ( 0.15299045, -1.17605023e+00,  0.981314

In [88]:
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib import cm
cmap = plt.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=min(Energy_over_mesh), vmax=max(Energy_over_mesh))
scalarMap = cm.ScalarMappable(norm=norm, cmap=cmap)
coloring = (255*scalarMap.to_rgba(Energy_over_mesh)).astype(int)

In [190]:
#Writing to PLY file
#Vertices
Normal_vertices = [(Vertex[i][0],Vertex[i][1],Vertex[i][2]) for i in range(len(Vertex))]
camera_vertices = [(camera_locations[i][0],camera_locations[i][1],camera_locations[i][2]) for i in range(len(camera_locations))]
Write_vertices = Normal_vertices + camera_vertices
#Faces
coloring = (255*scalarMap.to_rgba(Energy_over_mesh)).astype(int)
Colored_Mesh = [(3,mesh.elements[1].data[i][0][0],mesh.elements[1].data[i][0][1],mesh.elements[1].data[i][0][2],coloring[i][0],coloring[i][1],coloring[i][2]) for i in range(len(mesh.elements[1].data))]
num_vertices = len(Write_vertices)
num_faces = len(Colored_Mesh)
header_lines = "ply\nformat ascii 1.0\ncomment meshes colored according to function\nelement vertex {}\ncomment modified vertices\nproperty float x\nproperty float y\nproperty float z\nelement face {}\nproperty list uchar int vertex_indices\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nend_header\n".format(num_vertices,num_faces)
for i in range(num_vertices):
    header_lines = header_lines + str(Write_vertices[i]).replace(",","")[1:-1] + '\n'
for i in range(num_faces):
    header_lines = header_lines + str(Colored_Mesh[i]).replace(",","")[1:-1] + '\n'

In [191]:
with open("./testing.ply","w") as f:
    f.write(header_lines)

In [189]:
str(Colored_Mesh[0]).replace(",","")[1:-1] + '\n'

'3 304 2159 2160 214 85 109\n'

In [173]:
print(header_lines)

ply
format ascii 1.0
comment meshes colored according to function
element vertex 3705
comment modified vertices
property float x
property float y
property float z
element face 7314
property list uchar int vertex_indices
property uchar red
property uchar green
property uchar blue
end_header



In [168]:
str(Colored_Mesh[0])[1:-1]

'3, 304, 2159, 2160, 214, 85, 109'

In [157]:
#Vertices
Normal_vertices = [(Vertex[i][0],Vertex[i][1],Vertex[i][2],255,255,255) for i in range(len(Vertex))]
camera_vertices = [(camera_locations[i][0],camera_locations[i][1],camera_locations[i][2],0,255,0) for i in range(len(camera_locations))]
Write_vertices = Normal_vertices + camera_vertices
# Write_vertices = Normal_vertices
Write_vertices = np.array(Write_vertices,dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])
#Faces
# Write_vertices = mesh.elements[0].data
coloring = (255*scalarMap.to_rgba(Energy_over_mesh)).astype(int)


In [164]:
Colored_Mesh = [(3,mesh.elements[1].data[i][0][0],mesh.elements[1].data[i][0][1],mesh.elements[1].data[i][0][2],coloring[i][0],coloring[i][1],coloring[i][2]) for i in range(len(mesh.elements[1].data))]
# Colored_Mesh = np.array(Colored_Mesh,dtype=[('vertex_indices','i4', (3,)),('red', 'u1'), ('green', 'u1'),('blue', 'u1')])
# Colored_Mesh = mesh.elements[1].data

In [109]:
[mesh.elements[1].data[i][0][0],mesh.elements[1].data[i][0][1],mesh.elements[1].data[i][0][2]]

[3607, 2585, 3045]

In [112]:
mesh.elements[1].data[i][0]

array([3607, 2585, 3045], dtype=uint32)

In [143]:
vertex = np.array([(0, 0, 0),(0, 1, 1),(1, 0, 1),(1, 1, 0)],dtype=[('x', 'f4'), ('y', 'f4'),('z', 'f4')])
face = np.array([([0, 1, 2], 255, 255, 255),([0, 2, 3], 255,   0,   0),([0, 1, 3],   0, 255,   0),([1, 2, 3],   0,   0, 255)],dtype=[('vertex_indices', 'i4', (3,)),('red', 'u1'), ('green', 'u1'),('blue', 'u1')])

In [159]:
el = PlyData([PlyElement.describe(Write_vertices, 'vertex'),PlyElement.describe(Colored_Mesh, 'faces')])
# el = PlyData([PlyElement.describe(vertex, 'vertex'),PlyElement.describe(face, 'faces')])

In [123]:
mesh.elements[1].data

array([(array([ 304, 2159, 2160], dtype=uint32),),
       (array([223, 220,  11], dtype=uint32),),
       (array([ 205, 2085, 2150], dtype=uint32),), ...,
       (array([3609,  137,  139], dtype=uint32),),
       (array([3607, 1667, 2585], dtype=uint32),),
       (array([3607, 2585, 3045], dtype=uint32),)],
      dtype=[('vertex_indices', 'O')])

In [132]:
np.array(mesh.elements[0].data)

array([(-0.09222261, -0.06294695, 0.94313824),
       (-0.09519143,  0.03630431, 0.91076416),
       (-0.14368136,  0.02614488, 0.818953  ), ...,
       (-0.15763107, -0.18896167, 0.85484   ),
       (-0.15862265, -0.18500939, 0.8460188 ),
       (-0.15163262, -0.16250317, 0.80566436)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4')])

In [30]:
PlyData(
        [
            PlyElement.describe(
                vertex, 'vertex',
                comments=['tetrahedron vertices']
            ),
            PlyElement.describe(face, 'face')
        ],
        text=text, byte_order=byte_order,
        comments=['single tetrahedron with colored faces']
    )

In [160]:
el.write('./testing.ply')

In [33]:
camera_locations[0]

array([ 0.0567906, -0.608357 , -0.0560196])

In [121]:
Colored_Mesh

array([([ 304, 2159, 2160], 214, 85, 109),
       ([ 223,  220,   11], 209, 78, 114),
       ([ 205, 2085, 2150], 222, 96, 100), ...,
       ([3609,  137,  139], 207, 75, 116),
       ([3607, 1667, 2585], 183, 48, 138),
       ([3607, 2585, 3045], 188, 54, 133)],
      dtype=[('vertex_indices', '<i4', (3,)), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

In [41]:
Normal_vertices[-1]

[(0.0567906, -0.608357, -0.0560196, 0, 255, 0),
 (0.0500168, -0.472463, -0.126023, 0, 255, 0),
 (0.0455078, -0.320976, -0.181085, 0, 255, 0),
 (0.0425243, -0.165877, -0.213792, 0, 255, 0),
 (0.0440988, -0.00671206, -0.225977, 0, 255, 0),
 (0.248954, -0.40401, 2.05167, 0, 255, 0),
 (0.244254, -0.550675, 1.98657, 0, 255, 0),
 (0.235098, -0.686659, 1.90333, 0, 255, 0),
 (0.22309, -0.807659, 1.80426, 0, 255, 0),
 (0.211161, -0.914217, 1.68983, 0, 255, 0),
 (0.199795, -1.00635, 1.55919, 0, 255, 0),
 (0.188561, -1.07877, 1.41939, 0, 255, 0),
 (0.180905, -1.12772, 1.28148, 0, 255, 0),
 (0.164936, -1.16184, 1.1347, 0, 255, 0),
 (0.15299, -1.17605, 0.981314, 0, 255, 0),
 (0.137305, -1.16977, 0.825019, 0, 255, 0),
 (0.122764, -1.14309, 0.670893, 0, 255, 0),
 (0.11053, -1.09646, 0.523455, 0, 255, 0),
 (0.0986679, -1.03093, 0.384199, 0, 255, 0),
 (0.0860142, -0.948989, 0.255723, 0, 255, 0),
 (0.075486, -0.85086, 0.13861, 0, 255, 0),
 (0.065857, -0.737725, 0.0340774, 0, 255, 0),
 (0.0570106, -0.608

In [43]:
Write_vertices

array([(-0.09222261, -0.06294695,  0.94313824, 255, 255, 255),
       (-0.09519143,  0.03630431,  0.91076416, 255, 255, 255),
       (-0.14368136,  0.02614488,  0.81895298, 255, 255, 255), ...,
       ( 0.065857  , -0.737725  ,  0.0340774 ,   0, 255,   0),
       ( 0.0570106 , -0.608669  , -0.0558104 ,   0, 255,   0),
       ( 0.0512997 , -0.519745  , -0.104547  ,   0, 255,   0)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

In [93]:
mesh.elements[1].data

array([(array([ 304, 2159, 2160], dtype=uint32),),
       (array([223, 220,  11], dtype=uint32),),
       (array([ 205, 2085, 2150], dtype=uint32),), ...,
       (array([3609,  137,  139], dtype=uint32),),
       (array([3607, 1667, 2585], dtype=uint32),),
       (array([3607, 2585, 3045], dtype=uint32),)],
      dtype=[('vertex_indices', 'O')])

In [33]:
import numpy as np
from plyfile import PlyData, PlyElement
from tqdm import tqdm
from time import time
import json
import cv2
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import trimesh
######################################### FUNCTIONS ###########################################################

SAMPLE_SIZE = 10

def valid(l):
    for x in l:
        if x != 0:
            return True
    return False

def return_camera_info(file1,file2):
    '''
    Reads json file and returns camera parameters and locations
    '''
    f1 = open(file1, "r")
    data_RC = json.load(f1)['params']

    f2 = open(file2, "r")
    data_P = json.load(f2)['data']

    data_P = [x for x in data_P if x['height'] != 0 and valid(x['P']) ]
    files = [x['name'] for x in data_P]

    # Camera details
    P = np.array([np.array(x['P']).reshape([3, 4]) for x in data_P])
    R = np.array([np.array(x['R']).reshape([3, 3]) for x in data_RC])
    C = np.array([np.array(x['C']) for x in data_RC])
    K = np.matmul(P[:, :, :3], np.linalg.inv(R))

    return (P,R,C,K,files)

def fetch_mesh_coordinates(mesh_coords):
    '''
    Fetches coordinates,normal,area of the meshes
    '''
    p1 = np.array(list(mesh.elements[0].data[mesh_coords[0][0]]))
    p2 = np.array(list(mesh.elements[0].data[mesh_coords[0][1]]))
    p3 = np.array(list(mesh.elements[0].data[mesh_coords[0][2]]))
    a1 = p2-p1
    a2 = p3-p1
    cross_product = np.cross(a1,a2)
    norm_cross_product = np.linalg.norm(cross_product)
    #Normal vector
    n =  cross_product/norm_cross_product
    #Area
    area = (1/2)*norm_cross_product
    return ((p1,p2,p3),n,area)

def fetch_area_normal(pts):
    '''
    Fetches area and normal of the triangle formed by the points
    '''
    p1 = pts[0]
    p2 = pts[1]
    p3 = pts[2]
    a1 = p2-p1
    a2 = p3-p1
    cross_product = np.cross(a1,a2)
    norm_cross_product = np.linalg.norm(cross_product)
    #Normal vector
    n =  cross_product/norm_cross_product
    #Area
    area = (1/2)*norm_cross_product
    return (n,area)

def SignedVolume(a,b,c,d):
    return (1.0/6.0)*np.dot(np.cross(b-a,c-a),d-a)

def intersecting_mesh(p1,p2,p3,q1,q2):
    """Finds if vector formed by q1,q2 intersects the triangle made by p1,p2,p3. Returns 1 if there is intersection else 0"""
    A1 = np.sign(SignedVolume(q1,p1,p2,p3))
    A2 = np.sign(SignedVolume(q2,p1,p2,p3))
    B1 = np.sign(SignedVolume(q1,q2,p1,p2))
    B2 = np.sign(SignedVolume(q1,q2,p2,p3))
    B3 = np.sign(SignedVolume(q1,q2,p3,p1))
    if (A1!=A2) and (B1==B2==B3):
        return 1
    else:
        return 0
    
def visibility(mesh_index,camera_loc):
    '''Calculates visibilty for a single mesh based on intersecting meshes'''
    (p1,p2,p3),n,area=Mesh_info[mesh_index]
    q1 = camera_loc
    q2 = (p1+p2+p3)/3
    vector = vector = (q2-q1)/np.linalg.norm(q2-q1)
    #Calculates points on the vector joining
    Mesh_vector = q1+ np.array([[list(np.linalg.norm((mesh_centroid-q1),axis=1)),]*3]).T[:,:,0]*vector
    dist = np.linalg.norm(np.mean(Mesh_vertices[mesh_index,:],axis=0)-Mesh_vertices[mesh_index,0])    
    Search_mesh = np.where((np.abs(Mesh_vector[:,0]-mesh_centroid[:,0])<=dist) & (np.abs(Mesh_vector[:,1]-mesh_centroid[:,1])<=dist) & (np.abs(Mesh_vector[:,2]-mesh_centroid[:,2])<=dist))[0] 
    #Modifying q2 so that it is just outside the triangle
    q2 = q2 + 0.00001*(q2-q1)/np.linalg.norm(q2-q1)
    #Visibility function
    visibility = []
    main_mesh = mesh_index
    for mesh_index in Search_mesh:
        (p1,p2,p3),n,area = Mesh_info[mesh_index]
        visibility.append(intersecting_mesh(p1,p2,p3,q1,q2))
    a = Search_mesh[np.where(np.array(visibility)==1)[0]]
    if len(a)>=2:
        return 0
    else: 
        return 1

def calculate_visibility_mesh(camera_locations):
    '''Calculates visibility for all cameras and meshes, indexed as Camera_visibility[camera][mesh]'''
    start = time()
    Camera_visibility = []
    for camera in tqdm(range(len(camera_locations)),desc="Calculating visibility for given list of {} cameras".format(len(camera_locations))):
        Mesh_visibility = np.zeros(len(mesh_centroid))
        Mesh_list = np.arange(len(mesh_centroid))
        camera_loc = np.array(list(camera_locations[camera]))
        Mesh_vertices_calc = Mesh_vertices.copy()
        while len(Mesh_list)>=1:
    #         print("{} meshes left....".format(len(Mesh_list)))
            mesh_index = Mesh_list[0]
            a = visibility(mesh_index,camera_loc)
            #Find neighbouring meshes
            indices = np.unique(np.where((Mesh_vertices_calc==Mesh_vertices[mesh_index][0]) | (Mesh_vertices_calc==Mesh_vertices[mesh_index][1]) | (Mesh_vertices_calc==Mesh_vertices[mesh_index][2]))[0])
            #Update the visibility as the one of the selected mesh   
            Mesh_visibility[Mesh_list[indices]] = a
            #Update the mesh list
            Mesh_list = np.delete(Mesh_list, indices)
            #Update the vertices list
            Mesh_vertices_calc = np.delete(Mesh_vertices_calc, indices,axis=0)
        Camera_visibility.append(Mesh_visibility)
    end = time()
    print("Done in {}sec".format(end-start))
    return Camera_visibility

def Energy_function_calc(Vertex_gradient,texture_linear_0,sample_size=SAMPLE_SIZE):
    '''Finds Energy value for the whole mesh and camera'''
    #Calculates barycentric coordinates
    points = []
    for u in np.arange(0,1,1/sample_size):
        for v in np.arange(0,1-u,1/sample_size):
            w = 1-u-v
            points.append([u,v,w])
    points = np.array(points)
    h,w,_ = np.shape(Images[0])
    integration = 0
    Energy_over_mesh = np.zeros(len(Mesh_info))
    
    for camera_index in tqdm(range(len(camera_locations)),desc="Calculating photometric loss..."):
        f_x = K[camera_index][0,0]
        f_y = K[camera_index][1,1]
        visibility = Camera_visibility[camera_index]
        P_camera = P[camera_index]
        C = np.array(list(camera_locations[camera_index]))
        for mesh_id in range(len(Mesh_info)):
            if visibility[mesh_id]!=0:    
                X_j = Vertex_gradient[mesh.elements[1].data[mesh_id][0]] 
                n_j,A_j = fetch_area_normal(X_j)
                #P=uA+vB+wC
                X_u = np.dot(points,X_j)
                #Finds x = PX
                se = np.dot(P_camera,np.hstack((X_u,np.ones((len(X_u),1)))).T).T
                se = np.divide(se,se[:,-1][:,np.newaxis])
                #Texture term shd be added, term from image fetched
                image_term = np.linalg.norm(Images[camera_index][np.clip(se[:,1].astype(int),0,h-1),np.clip(se[:,0].astype(int),0,w-1)]-np.array(texture_linear_0[mesh_id]),axis=1)
                #For alpha term
                d = X_u - C
                d_z = np.linalg.norm(d,axis=1)
                alpha = (10**-10)*(f_x*f_y)*np.divide(d,d_z[:,np.newaxis]**3)
                #Final integeration over a single meshe
                temp_val = np.abs(A_j*np.dot(image_term,np.dot(alpha,n_j)))*visibility[mesh_id]
                Energy_over_mesh[mesh_id] = Energy_over_mesh[mesh_id] + temp_val
                integration += temp_val
    return integration,Energy_over_mesh

def Numerical_gradient_mesh(vertex_id,Vertex,texture_linear_0,diff=0.0000001,sample_size=SAMPLE_SIZE):
    '''
    Calculates numerical gradient for a single vertex
    Central diffference Method used: (f(x+diff)-f(x-diff))/2*diff
    Sample_size: Number of sampling barycentric coordinates in a triangle N = sample_size*(sample_size+1)/2
    Since only the meshes surronding the point gets affected, hence only the subtraction of energies
    of those points taken
    '''
    #Calculates barycentric coordinates
    points = []
    for u in np.arange(0,1,1/sample_size):
        for v in np.arange(0,1-u,1/sample_size):
            w = 1-u-v
            points.append([u,v,w])
    points = np.array(points)
    h,w,_ = np.shape(Images[0])
    affected_meshes = np.unique(np.where((Mesh_vertices==Vertex[vertex_id]))[0])
    gradient = np.array([0.0,0.0,0.0]) 
    for axis in range(3):
        Vertex_gradient = Vertex.copy()
        integration = 0
        for camera_index in range(len(camera_locations)):
            f_x = K[camera_index][0,0]
            f_y = K[camera_index][1,1]
            visibility = Camera_visibility[camera_index]
            P_camera = P[camera_index]
            C = np.array(list(camera_locations[camera_index]))
            for mesh_id in affected_meshes:
                #Positive perturbation f(x+diff)
                if visibility[mesh_id]!=0:    
                    Vertex_gradient[vertex_id][axis] = Vertex[vertex_id][axis]+diff
                    X_j = Vertex_gradient[mesh.elements[1].data[mesh_id][0]] 
                    n_j,A_j = fetch_area_normal(X_j)
                    #P=uA+vB+wC
                    X_u = np.dot(points,X_j)
                    #Finds x = PX
                    se = np.dot(P_camera,np.hstack((X_u,np.ones((len(X_u),1)))).T).T
                    se = np.divide(se,se[:,-1][:,np.newaxis])
                    #Texture term shd be added, term from image fetched
                    image_term = np.linalg.norm(Images[camera_index][np.clip(se[:,1].astype(int),0,h-1),np.clip(se[:,0].astype(int),0,w-1)]-np.array(texture_linear_0[mesh_id]),axis=1)
                    #For alpha term
                    d = X_u - C
                    d_z = np.linalg.norm(d,axis=1)
                    alpha = (10**-10)*(f_x*f_y)*np.divide(d,d_z[:,np.newaxis]**3)
                    #Final integeration over a single meshe
                    integration_pos = A_j*np.abs(np.dot(image_term,np.dot(alpha,n_j)))*visibility[mesh_id]

                    #Negative perturbation f(x-diff)
                    Vertex_gradient[vertex_id][axis] = Vertex[vertex_id][axis]-diff
                    X_j = Vertex_gradient[mesh.elements[1].data[mesh_id][0]] 
                    n_j,A_j = fetch_area_normal(X_j)
                    #P=uA+vB+wC
                    X_u = np.dot(points,X_j)
                    #Finds x = PX
                    se = np.dot(P_camera,np.hstack((X_u,np.ones((len(X_u),1)))).T).T
                    se = np.divide(se,se[:,-1][:,np.newaxis])
                    #Texture term shd be added, term from image fetched
                    image_term = np.linalg.norm(Images[camera_index][np.clip(se[:,1].astype(int),0,h-1),np.clip(se[:,0].astype(int),0,w-1)]-np.array(texture_linear_0[mesh_id]),axis=1)
                    #For alpha term
                    d = X_u - C
                    d_z = np.linalg.norm(d,axis=1)
                    alpha = (10**-10)*(f_x*f_y)*np.divide(d,d_z[:,np.newaxis]**3)
                    #Final integeration over a single meshe
                    integration_neg = A_j*np.abs(np.dot(image_term,np.dot(alpha,n_j)))*visibility[mesh_id]

                    #Final addition 
                    integration += (integration_pos-integration_neg)
        gradient[axis] = integration/(2*diff)
    return gradient

def write_to_PLY(Vertex_update,Energy_function):
    '''
    Writes the vertex and faces into PLY file with the help of Vertex_update. The meshes will
    have color according to the defined energy function over the meshes. Can be texture too
    Parameters:
        Vertex_update: Array of vertexes
        Energy_function: Value of energy over different meshes
    Returns:
        ply file named mesh_visualize.ply
    '''
    #Defining colormap
    cmap = plt.cm.get_cmap('plasma')
    norm = mpl.colors.Normalize(vmin=min(Energy_function), vmax=max(Energy_function))
    scalarMap = cm.ScalarMappable(norm=norm, cmap=cmap)
    coloring = (255*scalarMap.to_rgba(Energy_function)).astype(int)
    #Vertices
    Normal_vertices = [(Vertex_update[i][0],Vertex_update[i][1],Vertex_update[i][2]) for i in range(len(Vertex_update))]
    camera_vertices = [(camera_locations[i][0],camera_locations[i][1],camera_locations[i][2]) for i in range(len(camera_locations))]
    Write_vertices = Normal_vertices + camera_vertices
    #Faces
    coloring = (255*scalarMap.to_rgba(Energy_function)).astype(int)
    Colored_Mesh = [(3,mesh.elements[1].data[i][0][0],mesh.elements[1].data[i][0][1],mesh.elements[1].data[i][0][2],coloring[i][0],coloring[i][1],coloring[i][2]) for i in range(len(mesh.elements[1].data))]
    num_vertices = len(Write_vertices)
    #Writing to PLY file
    num_vertices = len(Write_vertices)
    num_faces = len(Colored_Mesh)
    header_lines = "ply\nformat ascii 1.0\ncomment meshes colored according to function\nelement vertex {}\ncomment modified vertices\nproperty float x\nproperty float y\nproperty float z\nelement face {}\nproperty list uchar int vertex_indices\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nend_header\n".format(num_vertices,num_faces)
    for i in range(num_vertices):
        header_lines = header_lines + str(Write_vertices[i]).replace(",","")[1:-1] + '\n'
    for i in range(num_faces):
        header_lines = header_lines + str(Colored_Mesh[i]).replace(",","")[1:-1] + '\n'
    with open("./mesh_visualize.ply","w") as f:
        f.write(header_lines)
        
def init_texture_coords():
    A, B, C = vertices_0[:, 0], vertices_0[:, 1], vertices_0[:, 2]
    A = np.hstack([A, np.ones([A.shape[0], 1], dtype=int)])
    B = np.hstack([B, np.ones([B.shape[0], 1], dtype=int)])
    C = np.hstack([C, np.ones([C.shape[0], 1], dtype=int)])

    start = time()

    texture_coords = np.zeros([mesh_0.faces.shape[0], len(image_files), 3, 2], dtype=np.float64)
    b_img_tex = np.zeros([mesh_0.faces.shape[0], len(image_files)], dtype=bool)

    for cam in range(len(image_files)):
        # print("%.2f%% Complete. Time Taken: %.2fs" % (100 * cam / len(image_files), time() - start))
        p = P[cam]
        img = images_0[cam]
        for i in range(vertices_0.shape[0]):
            if not visibility_0[cam][i]:
                continue
            a, b, c = A[i], B[i], C[i]
            a_, b_, c_ = p.dot(a.T), p.dot(b.T), p.dot(c.T)
            a_, b_, c_ = a_ / a_[-1], b_ / b_[-1], c_ / c_[-1]
            a__ = np.round(a_)
            b__ = np.round(b_)
            c__ = np.round(c_)
            if (0 <= int(a__[1]) < img.shape[0]) \
                    and (0 <= int(b__[1]) < img.shape[0]) \
                    and (0 <= int(c__[1]) < img.shape[0]) \
                    and (0 <= int(a__[0]) < img.shape[1]) \
                    and (0 <= int(b__[0]) < img.shape[1]) \
                    and (0 <= int(c__[0]) < img.shape[1]):
                b_img_tex[i][cam] = True
                texture_coords[i][cam][0] = a__[:2]
                texture_coords[i][cam][1] = b__[:2]
                texture_coords[i][cam][2] = c__[:2]
    return texture_coords, b_img_tex


def getTex(faceId, X_, u=-1., v=-1.):
    A, B, C = vertices_0[faceId]
    if u == -1 or v == -1:
        X = X_ - A
        B -= A
        C -= A
        v = (X[1] * B[0] - B[1] * X[0]) / (B[0] * C[1] - B[1] * C[0])
        u = (X[0] - v * C[0]) / B[0]
    res_tex = np.array([0., 0., 0.])
    w = 0
    for cam in range(len(images_0)):
        if not b_img_tex_0[faceId][cam]:
            continue
        a, b, c = texture_coords_0[faceId][cam]
        b, c = b - a, c - a
        coords = a + u * b + v * c
        d = X_ - CamCenter[cam]
        # w_temp = np.abs(K[cam][0][0] * K[cam][1][1] * np.dot(normals_0[faceId], d) / (np.linalg.norm(d) ** 3))
        w_temp = 1              # NOT WORKING WHEN W_TEMP IS COMPUTED PROPERLY
        res_tex += images_0[cam][int(coords[1])][int(coords[0])] * w_temp
        w += w_temp
    if w != 0:
        return res_tex / w, True
    return res_tex, False
    
    
def show_texture():
    points = []
    colors = []

    for i in range(mesh_0.faces.shape[0]):
        A, B, C = vertices_0[i]
        for u in range(0):
            for v in range(0, reso_0 - u):
                X = A + u / reso_0 * (B - A) + v / reso_0 * (C - A)
                tex_val, flag = getTex(i, X, u / reso_0, v / reso_0)
                if flag:
                    points.append(A + u / reso_0 * (B - A) + v / reso_0 * (C - A))
                    # colors.append(texture[i][u * reso + v] / wt[i][u * reso + v])
                    colors.append(tex_val)
    pcd = trimesh.PointCloud(vertices=points, colors=colors)
    print("Time taken: ", time() - start_0)
#     pcd.scene().show()
    pcd.export("mesh_textured.ply")
    print("Texture saved to file: mesh_textured.ply")
    
def init_texture():
    A, B, C = vertices_0[:, 0], vertices_0[:, 1], vertices_0[:, 2]
    A = np.hstack([A, np.ones([A.shape[0], 1], dtype=int)])
    B = np.hstack([B, np.ones([B.shape[0], 1], dtype=int)])
    C = np.hstack([C, np.ones([C.shape[0], 1], dtype=int)])

    start = time()

    texture = np.zeros([mesh_0.faces.shape[0], SAMPLE_SIZE * SAMPLE_SIZE, 3], dtype=np.float64)
    wt = np.zeros([mesh_0.faces.shape[0], SAMPLE_SIZE * SAMPLE_SIZE], dtype=float)
    print("Computing texture")
    for cam in tqdm(range(len(image_files)),desc="Calculating texture {} ".format(len(image_files))):
        # print("%.2f%% Complete. Time Taken: %.2fs" % (100 * cam / len(image_files), time() - start))
        p = P[cam]
        img = images_0[cam]
        # alpha = K[cam][0][0] * K[cam][1][1]
        for i in range(vertices_0.shape[0]):
            if not visibility_0[cam][i]:
                continue
            a, b, c = A[i], B[i], C[i]
            a_, b_, c_ = p.dot(a.T), p.dot(b.T), p.dot(c.T)
            a_, b_, c_ = a_ / a_[-1], b_/b_[-1], c_/c_[-1]
            for u in np.arange(0, 1, 1/SAMPLE_SIZE):
                for v in np.arange(0, 1 - u, 1/SAMPLE_SIZE):
                    # u, v, w
                    w = 1 - u - v
                    pos = np.round(u * a_ + v * b_ + w * c_)
                    if pos[0] >= 640 or pos[1] >= 480:
                        continue
                    # d = (a + u * b + v * c)[:3] - CamCenter[cam]
                    # wt_temp = np.abs(normals_0[i].dot(d) * alpha / (d[-1] ** 3))
                    wt_temp = 1
                    # if wt_temp == 0 or np.isnan(wt_temp):
                    #     continue
                    x_coord, y_coord = pos[1], pos[0]
                    wt[i][int(u * SAMPLE_SIZE * SAMPLE_SIZE + v * SAMPLE_SIZE)] += wt_temp
                    texture[i][int(u * SAMPLE_SIZE * SAMPLE_SIZE + v * SAMPLE_SIZE)] += img[int(x_coord)][int(y_coord)] * wt_temp
    return texture, wt


def show_texture_2(texture, wt):
    points = []
    colors = []

    for i in range(mesh_0.faces.shape[0]):
        A, B, C = vertices_0[i]
        for u in np.arange(0, 1, 1/SAMPLE_SIZE):
            for v in np.arange(0, 1 - u, 1/SAMPLE_SIZE):
                if wt[i][int(u * SAMPLE_SIZE * SAMPLE_SIZE + v * SAMPLE_SIZE)] == 0:
                    continue
                w = 1 - u - v
                X = u * A + v * B + w * C
                points.append(X)
                colors.append(texture[i][int(u * SAMPLE_SIZE * SAMPLE_SIZE + v * SAMPLE_SIZE)] / wt[i][int(u * SAMPLE_SIZE * SAMPLE_SIZE + v * SAMPLE_SIZE)])
    pcd = trimesh.PointCloud(vertices=points, colors=colors)
    # pcd.scene().show()
    pcd.export("mesh_textured_2.ply")
    

def linearize_texture(texture):
    texture_mesh = []
    for i in range(mesh_0.faces.shape[0]):
        texture_linear = []
        for u in np.arange(0, 1, 1/SAMPLE_SIZE):
            for v in np.arange(0, 1 - u, 1/SAMPLE_SIZE):
                texture_linear.append(texture[i][int(u * SAMPLE_SIZE * SAMPLE_SIZE + v * SAMPLE_SIZE)])
        texture_mesh.append(texture_linear)
    return texture_mesh
    
    


In [2]:
#########################################################################################################################
#Fetch initial scene data
P,R,camera_locations,K,files = return_camera_info("camera_params.test","images.test")
mesh = PlyData.read('./scene_dense_mesh_refine.ply')
#Load all images for each camera
operating_dir = "./datasets/templeRing/"
Images = [cv2.imread(operating_dir+files[i].split("/")[-1]) for i in range(len(camera_locations))]

#Extract mesh related information
Mesh_info = []
for i in tqdm(range(len(mesh.elements[1].data)),desc="Fetching mesh related information..."):
    Mesh_info.append(fetch_mesh_coordinates(mesh.elements[1].data[i]))
#Coordinates of vertex each mesh
Mesh_vertices = np.array([[i[0][0],i[0][1],i[0][2]]for i in Mesh_info])
#Centroid of mesh
mesh_centroid = np.mean(Mesh_vertices,axis=1)
#Array of vertices
Vertex = np.array([np.array(list(mesh.elements[0].data[i])) for i in range(len(mesh.elements[0].data))])

Fetching mesh related information...: 100%|██████████| 7314/7314 [00:03<00:00, 2390.06it/s]


In [3]:
start_0 = time()
CamCenter = camera_locations
image_files = [operating_dir+files[i].split("/")[-1] for i in range(len(camera_locations))]
mesh_0 = trimesh.load('scene_dense_mesh_refine.ply')
centroids_0 = mesh_0.vertices[mesh_0.faces].mean(axis=1)
vertices_0 = mesh_0.vertices[mesh_0.faces]
# print("TESTING IF EQL: ", (Mesh_vertices == vertices_0).all())
normals_0 = np.cross(vertices_0[:, 2] - vertices_0[:, 0], vertices_0[:, 1] - vertices_0[:, 0])
# images_0 = [mpl.image.imread(f) for f in image_files]
images_0 = Images
#Visibility table
Camera_visibility = calculate_visibility_mesh(camera_locations)

visibility_0 =  Camera_visibility

Calculating visibility for given list of 24 cameras: 100%|██████████| 24/24 [10:23<00:00, 25.96s/it]

Done in 623.0783743858337sec


In [4]:
visibility_0 =  Camera_visibility
# texture_coords_0, b_img_tex_0 = init_texture_coords()
texture_0, wt_0 = init_texture()
texture_linear_0 = linearize_texture(texture_0)
show_texture_2(texture_0, wt_0)

Calculating texture 24 :   0%|          | 0/24 [00:00<?, ?it/s]

Computing texture


Calculating texture 24 : 100%|██████████| 24/24 [06:47<00:00, 16.96s/it]


In [25]:
integeration,Energy_over_mesh = Energy_function_calc(Vertex,texture_linear_0)

100%|██████████| 24/24 [01:14<00:00,  3.10s/it]


In [32]:
Numerical_gradient_mesh(34,Vertex,texture_linear_0)

array([-0.12323645, -0.06171477, -0.01915466])

In [27]:
Energy_over_mesh

array([ 24.99285963,  78.74956628,  33.14429977, ...,  44.09043218,
       170.6960138 , 149.55994153])

In [19]:
texture_linear_0 = linearize_texture(texture_0)

In [21]:
np.array(texture_linear_0[144])

array([[ 824.,  933., 1170.],
       [ 856.,  982., 1218.],
       [ 887., 1014., 1246.],
       [ 904., 1023., 1255.],
       [ 914., 1034., 1256.],
       [ 904., 1031., 1262.],
       [ 918., 1054., 1275.],
       [ 906., 1037., 1258.],
       [ 864., 1003., 1221.],
       [ 867.,  992., 1210.],
       [ 865.,  987., 1208.],
       [ 884.,  996., 1221.],
       [ 924., 1044., 1268.],
       [ 933., 1048., 1273.],
       [ 923., 1034., 1261.],
       [ 924., 1048., 1272.],
       [ 914., 1038., 1253.],
       [ 895., 1011., 1228.],
       [ 892., 1028., 1239.],
       [ 889., 1008., 1242.],
       [ 921., 1044., 1264.],
       [ 931., 1047., 1278.],
       [ 923., 1043., 1265.],
       [ 926., 1039., 1253.],
       [ 923., 1047., 1256.],
       [ 916., 1029., 1248.],
       [ 910., 1023., 1238.],
       [ 948., 1069., 1297.],
       [ 947., 1073., 1292.],
       [ 943., 1060., 1278.],
       [ 926., 1045., 1265.],
       [ 937., 1058., 1264.],
       [ 922., 1039., 1251.],
       [ 9

In [8]:
np.array(texture_linear_0).reshape(len(Mesh_info),114,3)

ValueError: cannot reshape array of size 1206810 into shape (7314,114,3)

In [23]:
integeration,Energy_over_mesh = Energy_function_calc()

Calculating energy function...:   0%|          | 0/24 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (55,) (3,) 

In [43]:
def gradient_descent(Epochs,Vertices_grad,texture_linear_0,learning_rate):
    loss,_ = Energy_function_calc(Vertex,texture_linear_0)
    epoch = 0
    for epoch in tqdm(range(Epochs),position=0, leave=True,desc="Epoch{} photometric loss:{}".format(epoch,loss)):
        for vertex_id in range(len(Vertices_grad)):
            Vertices_grad[vertex_id] = Vertices_grad[vertex_id] - learning_rate*Numerical_gradient_mesh(vertex_id,Vertices_grad,texture_linear_0)
        if epoch%1==0:
            loss,_= Energy_function_calc(Vertex,texture_linear_0)
    return Vertices_grad




def Latent_Training(step_size,dim,Epochs,batch_size,df,base_params):
    base_mui, base_bu, base_bi = base_params
    #Initialize values for bu,b'i,miu
    #Since we are given there are 10k users and 10k movie id
    pu = np.ones((10000,dim)) / dim
    qi = np.ones((10000,dim)) / dim
    #Taking miu from baseline
    miu = base_mui
    bu = base_bu
    bi = base_bi
    
    num_batches = df.shape[0]//batch_size + 1
    for epoch in range(Epochs):
        for batch in range(num_batches):
            bui = []
            X = minibatch(batch,df,batch_size)
            userid =  X.iloc[:,0]
            movieid = X.iloc[:,1]
            bui.append((miu+bu[userid]+bi[movieid]))
            bui = np.ravel(np.array(bui).T)
            pu_temp = pu
            qi_temp = qi
            for i in range(len(X)):
                userid =  X.iloc[i,0]
                movieid = X.iloc[i,1]
                prod = np.dot(pu[userid],qi[movieid])
                pu_temp[userid]  += step_size*qi[movieid]*(X.iloc[i,2]-bui[i]-prod)
                qi_temp[movieid] += step_size*pu[userid]*(X.iloc[i,2]-bui[i]-prod)
            pu = pu_temp
            qi = qi_temp
            
            if batch %500 ==0:
#                 print(pu,qi)
                print("Epoch:{} Batch:{} ------ Training Error:{}".format(epoch,batch,loss((miu,bu,bi,pu,qi),X)))
    return pu, qi

In [44]:
Vertex_new = gradient_descent(15,Vertex,texture_linear_0,0.001)

100%|██████████| 3681/3681 [00:21<00:00, 170.69it/s]      | 0/15 [00:00<?, ?it/s]

Epoch0 photometric loss:3.390880306855681:   7%|▋         | 1/15 [02:00<28:00, 120.05s/it]


KeyboardInterrupt: 

In [22]:
Energy_over_mesh

array([-3.17664625, -4.26910531, -1.34802177, ..., -4.83092415,
       -9.51784758, -8.13370732])

In [17]:
texture_0[532]

array([[1244., 1691., 2005.],
       [1071., 1482., 1784.],
       [ 980., 1362., 1654.],
       [ 921., 1282., 1571.],
       [ 859., 1188., 1460.],
       [ 711.,  992., 1252.],
       [ 566.,  821., 1061.],
       [ 563.,  801., 1030.],
       [ 484.,  700.,  915.],
       [ 440.,  634.,  838.],
       [1208., 1652., 1949.],
       [1058., 1464., 1753.],
       [ 883., 1240., 1530.],
       [ 838., 1168., 1441.],
       [ 779., 1086., 1352.],
       [ 636.,  897., 1151.],
       [ 539.,  782., 1020.],
       [ 508.,  735.,  950.],
       [ 465.,  671.,  891.],
       [   0.,    0.,    0.],
       [1149., 1564., 1860.],
       [1004., 1391., 1682.],
       [ 861., 1207., 1486.],
       [ 793., 1113., 1372.],
       [ 737., 1012., 1271.],
       [ 583.,  835., 1087.],
       [ 524.,  761.,  996.],
       [ 496.,  705.,  935.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [1066., 1462., 1751.],
       [ 968., 1334., 1618.],
       [ 831., 1164., 1424.],
       [ 7

In [18]:
wt_0[532]

array([21., 21., 21., 21., 21., 21., 21., 21., 21., 21., 21., 21., 21.,
       21., 21., 21., 21., 21., 21.,  0., 21., 21., 21., 21., 21., 21.,
       21., 21.,  0.,  0., 21., 21., 21., 21., 21., 21., 21.,  0.,  0.,
        0., 21., 21., 21., 21., 21., 21.,  0.,  0.,  0.,  0., 21., 21.,
       21., 21., 21.,  0.,  0.,  0.,  0.,  0., 21., 21., 21., 21.,  0.,
        0.,  0.,  0.,  0.,  0., 21., 21., 21.,  0.,  0.,  0.,  0.,  0.,
        0.,  0., 21., 21.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 21.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [25]:
points = []
for u in np.arange(0,1,1/14):
    for v in np.arange(0,1-u,1/14):
        w = 1-u-v
        points.append([u,v,w])
points = np.array(points)


In [ ]:
texture_0[mesh_id][int(u * SAMPLE_SIZE * SAMPLE_SIZE + v * SAMPLE_SIZE)]

In [28]:
len(points)

114

In [26]:
texture_0[23]

array([[2939., 4085., 4540.],
       [2868., 4034., 4550.],
       [2786., 3951., 4509.],
       [2605., 3718., 4311.],
       [2480., 3518., 4141.],
       [1880., 2800., 3471.],
       [1028., 1807., 2445.],
       [ 554., 1170., 1758.],
       [ 403.,  878., 1391.],
       [ 527.,  852., 1289.],
       [2896., 4087., 4534.],
       [2774., 3960., 4484.],
       [2699., 3838., 4406.],
       [2544., 3615., 4234.],
       [2127., 3082., 3741.],
       [1269., 2098., 2749.],
       [ 640., 1315., 1908.],
       [ 397.,  928., 1466.],
       [ 490.,  863., 1334.],
       [   0.,    0.,    0.],
       [2794., 3979., 4493.],
       [2749., 3897., 4473.],
       [2567., 3642., 4251.],
       [2281., 3268., 3913.],
       [1616., 2486., 3161.],
       [ 728., 1424., 2039.],
       [ 395.,  962., 1520.],
       [ 456.,  885., 1365.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [2791., 3942., 4504.],
       [2674., 3764., 4358.],
       [2404., 3412., 4040.],
       [17